In [1]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 194kB 16.8MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 12.8MB 240kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [5]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [6]:
def download_file_from_google_drive(id,destination):
  URL = "https://docs.google.com/uc?export=download"
  session = requests.Session()
  response = session.get(URL, params = {'id' :id},stream = True)
  token = get_confirm_token(response)
  if token:
    params = {'id' : id, 'confirm':token}
    response = session.get(URL,params = params, stream = True)
    save_response_content(response, destination)
def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None
def save_response_content(response,destination):
  CHUNK_SIZE = 32768
  with open(destination, "wb") as f:
    for chunk in response.iter_content(CHUNK_SIZE):
      if chunk:
        f.write(chunk)

In [7]:
file_id = '1--w4pqXsYeLdUzjROb3oNV3bXjhWyiW0'
destination = './fruits_vegetables_classifier.pk1'
download_file_from_google_drive(file_id,destination)

##Classifier for Fruits and Vegetables 

Folgende Früchte/Gemüsesorten werden erkannt:


*   Orange
*   Banana
*   Apple
*   Grapefruit
*   Tomato
*   Cucumber
*   Cherries
*   Broccoli
*   Carrot
*   Pineapple
*   Lemon
*   Grapes
*   Pear
*   Raspberries
*   Watermelon
*   Radish
*   Potato
*   Kiwi
*   Strawberries
*   Cranberries























In [8]:
path = Path()
inference = load_learner(path/'fruits_vegetables_classifier.pk1',cpu = True)

In [9]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_p1 = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_p1.clear_output()
  with out_p1: display(img.to_thumb(128,128))
  pred,pred_idx,probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
  
btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your fruit/vegetable!'),
              btn_upload,btn_run,out_p1,lbl_pred])